# Requirement

You are a ML engineer. You need to implement a model that classifies digits in the MNIST dataset. Most of the code has been implemented, but there are still parts to be implemented. The CNN model needs to be further implemented. Optimization is also required to increase inference performance. You might need hyperparameter tuning.

# Constraints
It's not allowed to add or delete cells. You can't change the `DO NOT CHANGE` cells. Only `CHANGEABLE` cells can be changed. Of course, you can add or delete them during competitions. But, they must be returned to proper condition prior to the end of the competiton. You may lose points if you change cells that are not allowed or solve it usnig a illegal method.

In [ ]:
# Cell 1 - Import packages
# DO NOT CHANGE

import torch
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch import nn
import torch.nn.init
from torch.nn import functional as F
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
# Cell 2 - Hyperparameters
# CHANGEABLE

device = 'cpu'
torch.manual_seed(111)
learning_rate = 1.0
training_epochs = 100
batch_size = 1
dropout_ratio = 1.0
is_shutffle = False
criterion = nn.CrossEntropyLoss()
classes = ('1', '2', '3', '4', '5', '6','7','8','9')

In [ ]:
# Cell 3 - Data preprocessing
# DO NOT CHANGE

transformer = transforms.Compose([transforms.ToTensor()])

trainset = dsets.MNIST(root='wsi_vision_data/',
                         train=True,
                         transform=transformer,
                         download=True)

testset = dsets.MNIST(root='wsi_vision_data/',
                         train=False,
                         transform=transformer,
                         download=True)

train_loader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=batch_size,
                                          shuffle=is_shutffle,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=batch_size,
                                          shuffle=is_shutffle,
                                          drop_last=True)

In [ ]:
# Cell 4 - Visualization
# CHANGEABLE

def show_samples(img):
    pass

In [ ]:
# Cell 5 - CNN Model implement
# CHANGEABLE

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Conv layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Fully-connected
        self.fc1 = nn.Linear(64*7*7, 32)
        self.fc2 = nn.Linear(32, 10)
        self.dropout1 = nn.Dropout(dropout_ratio)

    def forward(self, x):
        # Conv1
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        # Conv2
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        # FC
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        out = self.fc2(x)

        return out

In [ ]:
# Cell 6 - Parameter setting
# DO NOT CHANGE

model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = criterion.to(device)
total_batch_size = len(train_loader)

In [ ]:
# Cell 7 - Trainer
# DO NOT CHANGE

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, Y)
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch_size

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:
# Cell 8 - Evaluation
# DO NOT CHANGE

y_trues = np.array([])
y_preds = np.array([])

model.eval()
with torch.no_grad():
    for X, Y in test_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        prediction = model(X)
        pred = torch.argmax(prediction, 1)
        
        y_trues = np.append(y_trues, Y.cpu())
        y_preds = np.append(y_preds, pred.cpu())

print('accuracy:', accuracy_score(y_trues, y_preds, normalize=True))
print('precision:', precision_score(y_trues, y_preds, average='macro'))
print('recall:', recall_score(y_trues, y_preds, average='macro'))
print('F1-score:', f1_score(y_trues, y_preds, average='macro'))